In [ ]:
import sys
import pandas as pd
import numpy as np

# Quick setup
from src.utils.notebook_setup import setup_notebook_environment
dbs, logger = await setup_notebook_environment()

# Now ready to work
logger.info("Starting model training...")